<a href="https://colab.research.google.com/github/carlosks/carlosks/blob/main/Camuflagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import randon
inport numpy as np

# Define the target background color (you can change these values as needed)
target_color = (100,150,200)

# Define parameters for the genetic algorithm
population_size = 25
num_generations = 100

# Mutation probability
mutation_rate = 0.01

# Mutation intensity as percentage
mutation_intensity = 0.001

# Function to calculate the fitness of an individual color
def calculate_fitness(color):
    return sum(abs(color[i] - target_color[i]) for i in range(3))

# Generate a random opulation of colors
def generate_random_population (size):
    return [(randon.randint(0, 255),random.randint(0, 255), random.randint(0, 255)) for _ in range(size)]